In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
pip install keras_applications

In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.optimizers import Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam

In [ ]:
from __future__ import print_function

import numpy as np
import warnings

from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs

from keras_applications.imagenet_utils import _obtain_input_shape

WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'


In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.engine import  Input, Model
from keras.optimizers import SGD
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time

In [ ]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((224, 224))
image_size = 0
directory_root = '../input/ddaattaa/'
width=224
height=224
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)
            for image in plant_disease_image_list[:500 ]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

In [ ]:
image_size = len(image_list)
image_size

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [ ]:
print(label_binarizer.classes_)

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16)

In [ ]:
#First to split to train, test and then split train again into validation and train. Something like this 
x_train, x_test, y_train, y_test  = train_test_split(np_image_list, image_labels,test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x


def ResNet50(include_top=True, weights='imagenet',
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=6):
    """Instantiates the ResNet50 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 197.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 10:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 10')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

       
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
  
   
    x = AveragePooling2D((7, 7), name='avg_pool')(x)
    

    if include_top:
        x = Flatten()(x)
        x=Dense(512,activation='relu',name='fc160')(x)
        x=Dense(256,activation='relu',name='fc12')(x)
        x = Dense(classes, activation='softmax', name='fc10')(x)
        
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models',
                                    md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc10')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model
    

In [ ]:
if __name__ == '__main__':
    model = ResNet50(include_top=True, weights=None)
    model.summary()


In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")


In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:

history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
plt.style.use('ggplot')
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)
#Train and validation accuracy
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend(loc="lower right")
plt.figure()
#Train and validation loss
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, loss, 'b', label='Training loss')
m=0.8
plt.fill_between(acc,val_acc,alpha=1.0,where=(m>0.8),interpolate='true')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
loss, accuracy = model.evaluate(x_test, y_test)

print(loss,accuracy)

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Testing accuracy')
plt.title('Testing accuracy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Testing loss')
plt.title('Testing loss')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(x_test)

#getting labels
y_pred_labels = np.argmax(y_pred,axis = 1)
y_true = np.argmax(y_test,axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc

cm = confusion_matrix(y_true,y_pred_labels)
cm = cm.astype(np.float) / cm.sum(axis=1)[:, np.newaxis]

In [ ]:
import pandas as pd
import seaborn as sns
from itertools import cycle

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [ ]:
#Transform to df for easier plotting
cm_df = pd.DataFrame(cm, index = label_binarizer.classes_,
                     columns = label_binarizer.classes_
                    )

In [ ]:
plt.figure(figsize = (5,5))
sns.heatmap(cm_df, annot = True,cmap='Blues',cbar=False)
plt.title('Paddy Classify')
plt.ylabel('True class')
plt.xlabel('Prediction class')
plt.show()

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(classification_report(y_true,y_pred_labels))

In [ ]:
cm_un = confusion_matrix(y_true,y_pred_labels)
cm_un = cm_un.astype(np.float)

In [ ]:
new_label = label_binarizer.classes_


In [ ]:
y_pred_ravel = y_pred.ravel()
lw = 2

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(new_class):
    fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_pred[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
colors = cycle(['red', 'green','black','purple','brown','orange'])
for i, color in zip(range(new_class), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0}'''.format(new_label[i]))
    

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
SGD

In [ ]:
opt1 = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt1,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
print("[INFO] Calculating model accuracy")
loss, accuracy = model.evaluate(x_test, y_test)

print(loss,accuracy)

In [ ]:
plt.style.use('ggplot')
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)
#Train and validation accuracy
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend(loc="lower right")
plt.figure()
#Train and validation loss
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, loss, 'b', label='Training loss')
m=0.8
plt.fill_between(acc,val_acc,alpha=1.0,where=(m>0.8),interpolate='true')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Testing accuracy')
plt.title('Testing accuracy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Testing loss')
plt.title('Testing loss')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(x_test)

#getting labels
y_pred_labels = np.argmax(y_pred,axis = 1)
y_true = np.argmax(y_test,axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc

cm = confusion_matrix(y_true,y_pred_labels)
cm = cm.astype(np.float) / cm.sum(axis=1)[:, np.newaxis]

In [ ]:
import pandas as pd
import seaborn as sns
from itertools import cycle

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [ ]:
#Transform to df for easier plotting
cm_df = pd.DataFrame(cm, index = label_binarizer.classes_,
                     columns = label_binarizer.classes_
                    )

In [ ]:
plt.figure(figsize = (5,5))
sns.heatmap(cm_df, annot = True,cmap='Blues',cbar=False)
plt.title('Paddy Classify')
plt.ylabel('True class')
plt.xlabel('Prediction class')
plt.show()

In [ ]:
#Evaluation of Model - Confusion Matrix Plot
import os
import itertools
from subprocess import check_output
def plot_confusion_matrix(final_cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap='Greys'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(final_cm)
    plt.figure(figsize = (7,7))

    plt.imshow(final_cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes)-1)
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = final_cm.max() / 2.
    for i, j in itertools.product(range(final_cm.shape[0]), range(final_cm.shape[1])):
        plt.text(j, i, format(final_cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if final_cm[i, j] > thresh else "black")

    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.tight_layout()


# Compute confusion matrix
cnf_matrix = confusion_matrix(y_true,y_pred_labels)
cnf_matrix = np.delete(cnf_matrix,(3), axis=1)
cnf_matrix = np.delete(cnf_matrix,(3), axis=0)


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes= label_binarizer.classes_,
                      title='Confusion matrix, without normalization')

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(classification_report(y_true,y_pred_labels))

In [ ]:
cm_un = confusion_matrix(y_true,y_pred_labels)
cm_un = cm_un.astype(np.float)

In [ ]:
new_label = label_binarizer.classes_


In [ ]:
y_pred_ravel = y_pred.ravel()
lw = 2

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(new_class):
    fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_pred[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
colors = cycle(['red', 'green','black','purple','brown','orange'])
for i, color in zip(range(new_class), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0}'''.format(new_label[i]))
    

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
RMSProp

In [ ]:
opt2 = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt2,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
print("[INFO] Calculating model accuracy")
loss, accuracy = model.evaluate(x_test, y_test)

print(loss,accuracy)

In [ ]:
y_pred = model.predict(x_test)

#getting labels
y_pred_labels = np.argmax(y_pred,axis = 1)
y_true = np.argmax(y_test,axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc

cm = confusion_matrix(y_true,y_pred_labels)
cm = cm.astype(np.float) / cm.sum(axis=1)[:, np.newaxis]

In [ ]:
import pandas as pd
import seaborn as sns
from itertools import cycle

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [ ]:
#Transform to df for easier plotting
cm_df = pd.DataFrame(cm, index = label_binarizer.classes_,
                     columns = label_binarizer.classes_
                    )

In [ ]:
plt.figure(figsize = (5,5))
sns.heatmap(cm_df, annot = True,cmap='Blues',cbar=False)
plt.title('Paddy Classify')
plt.ylabel('True class')
plt.xlabel('Prediction class')
plt.show()

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(classification_report(y_true,y_pred_labels))

In [ ]:
ADAGRAAD

In [ ]:
opt3 = Adagrad(lr=0.0001, epsilon=1e-08, decay=0.0)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt3,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
print("[INFO] Calculating model accuracy")
loss, accuracy = model.evaluate(x_test, y_test)

print(loss,accuracy)

In [ ]:
plt.style.use('ggplot')
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)
#Train and validation accuracy
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend(loc="lower right")
plt.figure()
#Train and validation loss
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, loss, 'b', label='Training loss')
m=0.8
plt.fill_between(acc,val_acc,alpha=1.0,where=(m>0.8),interpolate='true')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Testing accuracy')
plt.title('Testing accuracy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Testing loss')
plt.title('Testing loss')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(x_test)

#getting labels
y_pred_labels = np.argmax(y_pred,axis = 1)
y_true = np.argmax(y_test,axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc

cm = confusion_matrix(y_true,y_pred_labels)
cm = cm.astype(np.float) / cm.sum(axis=1)[:, np.newaxis]

In [ ]:
import pandas as pd
import seaborn as sns
from itertools import cycle

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [ ]:
#Transform to df for easier plotting
cm_df = pd.DataFrame(cm, index = label_binarizer.classes_,
                     columns = label_binarizer.classes_
                    )

In [ ]:
plt.figure(figsize = (5,5))
sns.heatmap(cm_df, annot = True,cmap='Blues',cbar=False)
plt.title('Paddy Classify')
plt.ylabel('True class')
plt.xlabel('Prediction class')
plt.show()

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(classification_report(y_true,y_pred_labels))

In [ ]:
new_class=6
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(new_class):
    fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_pred[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
colors = cycle(['red', 'green','black','purple','brown','orange'])
for i, color in zip(range(new_class), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0}'''.format(new_label[i]))
    

plt.plot([0, 1], [0, 1], 'k--',lw=lw)
plt.xlim([0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
ADADELTA

In [ ]:
opt4 = Adadelta(lr=0.01, epsilon=1e-08, decay=0.0)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt4,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
print("[INFO] Calculating model accuracy")
loss, accuracy = model.evaluate(x_test, y_test)

print(loss,accuracy)

In [ ]:
plt.style.use('ggplot')
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)
#Train and validation accuracy
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend(loc="lower right")
plt.figure()
#Train and validation loss
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, loss, 'b', label='Training loss')
m=0.8
plt.fill_between(acc,val_acc,alpha=1.0,where=(m>0.8),interpolate='true')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Testing accuracy')
plt.title('Testing accuracy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Testing loss')
plt.title('Testing loss')
plt.legend()
plt.show()

In [ ]:
ADAMAX

In [ ]:
opt5 = Adamax(lr=0.01, epsilon=1e-08, decay=0.0)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt5,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
print("[INFO] Calculating model accuracy")
loss, accuracy = model.evaluate(x_test, y_test)
print(loss,accuracy)

In [ ]:
plt.style.use('ggplot')
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)
#Train and validation accuracy
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend(loc="lower right")
plt.figure()
#Train and validation loss
plt.fill_between(acc,val_acc,alpha=1.0)
plt.plot(epochs, loss, 'b', label='Training loss')
m=0.8
plt.fill_between(acc,val_acc,alpha=1.0,where=(m>0.8),interpolate='true')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Testing accuracy')
plt.title('Testing accuracy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Testing loss')
plt.title('Testing loss')
plt.legend()
plt.show()

In [ ]:
opt6 = Nadam(lr=0.01, epsilon=1e-08, decay=0.0)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt6,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
print("[INFO] Calculating model accuracy")
loss, accuracy = model.evaluate(x_test, y_test)
print(loss,accuracy)